# Session 4: Function Calling & Tool Use

**Duration**: 75 minutes  
**Difficulty**: Intermediate

## Learning Objectives

By the end of this session, you will:
- 🎯 Understand what function calling is and why it's powerful
- 🎯 Define function schemas for OpenAI
- 🎯 Implement custom tools for LLMs to use
- 🎯 Handle function call responses and errors
- 🎯 Build multi-tool systems
- 🎯 Validate and sanitize function inputs
- 🎯 Make SupportGenie action-capable

## 📚 What You'll Build

**SupportGenie v0.4**: Action-taking customer support agent
- Creates support tickets
- Looks up order status
- Checks account information
- **From chatbot → agent that takes actions!**

## Part 0: Setup

Install required packages and set up API keys.

In [ ]:
# Install required packages
!pip install openai python-dotenv -q

print("✅ Packages installed!")

In [ ]:
import os
import json
import random
import re
from openai import OpenAI

# Set up API key (Google Colab method)
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    # Fallback for local development
    from getpass import getpass
    if 'OPENAI_API_KEY' not in os.environ:
        os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')
    print("✅ API key loaded")

# Initialize client
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

print("\n🚀 Ready to start!")

## Part 1: What is Function Calling?

### The Problem

Traditional LLMs can only generate text:

```
User: "Create a support ticket for my broken laptop"
LLM: "I've created a support ticket for you."  ❌ (It didn't actually!)
```

### The Solution: Function Calling

LLMs can **request** to call functions, then you execute them:

```
User: "Create a support ticket for my broken laptop"
LLM: {
  "function": "create_ticket",
  "arguments": {
    "subject": "Broken laptop",
    "priority": "high"
  }
}

You: Execute create_ticket() → Returns ticket #12345

LLM: "I've created support ticket #12345 for your broken laptop."  ✅ (Actually did it!)
```

## Part 2: Your First Function - Order Status Lookup

In [ ]:
# Step 1: Define the function
def get_order_status(order_id: str) -> dict:
    """
    Look up the status of an order.
    
    Args:
        order_id: Order ID (format: ORD-XXXXX)
    
    Returns:
        Order status information
    """
    # Mock database lookup
    orders_db = {
        "ORD-12345": {
            "status": "shipped",
            "tracking": "1Z999AA1234567890",
            "estimated_delivery": "2024-12-30",
            "items": ["Laptop", "Mouse"]
        },
        "ORD-67890": {
            "status": "processing",
            "tracking": None,
            "estimated_delivery": "2024-12-31",
            "items": ["Keyboard"]
        }
    }
    
    order = orders_db.get(order_id)
    if not order:
        return {"error": f"Order {order_id} not found"}
    
    return order

# Test the function
print("Testing get_order_status function:")
print(json.dumps(get_order_status("ORD-12345"), indent=2))

In [ ]:
# Step 2: Define the function schema for OpenAI
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_order_status",
            "description": """Get the current status of a customer order.
            
            Use this when the customer asks about:
            - Order status
            - Shipping updates
            - Delivery tracking
            - "Where is my order?"
            
            Requires a valid order ID (format: ORD-XXXXX)""",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The order ID (e.g., ORD-12345)"
                    }
                },
                "required": ["order_id"]
            }
        }
    }
]

print("✅ Function schema defined")
print("\nSchema:")
print(json.dumps(tools[0], indent=2))

In [ ]:
# Step 3: Make API call with tools
messages = [
    {"role": "user", "content": "What's the status of order ORD-12345?"}
]

# Call API with tools
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    tools=tools,
    tool_choice="auto"  # Let LLM decide
)

response_message = response.choices[0].message

print("LLM Response:")
print(f"Has tool calls: {bool(response_message.tool_calls)}")

if response_message.tool_calls:
    tool_call = response_message.tool_calls[0]
    print(f"\n📞 LLM wants to call: {tool_call.function.name}")
    print(f"Arguments: {tool_call.function.arguments}")

In [ ]:
# Step 4: Execute the function and send result back to LLM
if response_message.tool_calls:
    tool_call = response_message.tool_calls[0]
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    
    # Execute the function
    if function_name == "get_order_status":
        result = get_order_status(**function_args)
    
    print("✅ Function executed")
    print(f"Result: {json.dumps(result, indent=2)}")
    
    # Add messages to history
    messages.append(response_message)
    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps(result)
    })
    
    # Get final response from LLM
    final_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    
    print("\n🤖 Final Answer:")
    print(final_response.choices[0].message.content)

## Part 3: Complete Function Calling Helper

Let's create a reusable function for the complete flow.

In [ ]:
def execute_function_call(messages, tools, available_functions):
    """
    Complete function calling flow
    
    Args:
        messages: Conversation history
        tools: Function definitions
        available_functions: Dict mapping names to actual functions
    
    Returns:
        Final response from LLM
    """
    # Initial API call
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    
    # Check if function call is needed
    if not response_message.tool_calls:
        return response_message.content
    
    # Add assistant's function call request to messages
    messages.append(response_message)
    
    # Execute each function call
    for tool_call in response_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        print(f"📞 Calling: {function_name}({function_args})")
        
        # Execute the function
        function_to_call = available_functions[function_name]
        function_result = function_to_call(**function_args)
        
        print(f"✅ Result: {function_result}\n")
        
        # Add function result to messages
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(function_result)
        })
    
    # Get final response with function results
    final_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    
    return final_response.choices[0].message.content

# Test it
messages = [
    {"role": "user", "content": "Check the status of order ORD-67890"}
]

available_functions = {
    "get_order_status": get_order_status
}

answer = execute_function_call(messages, tools, available_functions)
print(f"🤖 {answer}")

## Part 4: Multiple Tools

Let's add more tools to our system.

In [ ]:
# Define additional functions

def create_support_ticket(subject: str, description: str, priority: str = "medium") -> dict:
    """Create a support ticket"""
    ticket_id = f"TKT-{random.randint(10000, 99999)}"
    
    return {
        "ticket_id": ticket_id,
        "subject": subject,
        "description": description,
        "priority": priority,
        "status": "open",
        "created_at": "2024-12-29T10:00:00Z"
    }

def check_account_info(customer_email: str) -> dict:
    """Check customer account information"""
    # Mock data
    accounts = {
        "john@email.com": {
            "name": "John Smith",
            "member_since": "2023-01-15",
            "total_orders": 12,
            "rewards_points": 450,
            "tier": "Gold"
        },
        "jane@email.com": {
            "name": "Jane Doe",
            "member_since": "2024-06-20",
            "total_orders": 3,
            "rewards_points": 100,
            "tier": "Silver"
        }
    }
    
    account = accounts.get(customer_email)
    if not account:
        return {"error": "Account not found"}
    
    return account

print("✅ Functions defined")

# Test them
print("\nTest create_support_ticket:")
print(json.dumps(create_support_ticket("Broken product", "My laptop screen is cracked", "high"), indent=2))

print("\nTest check_account_info:")
print(json.dumps(check_account_info("john@email.com"), indent=2))

In [ ]:
# Define complete tool set
multi_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_order_status",
            "description": "Get order status and tracking information",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "Order ID (ORD-XXXXX)"
                    }
                },
                "required": ["order_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_support_ticket",
            "description": "Create a new support ticket for customer issues",
            "parameters": {
                "type": "object",
                "properties": {
                    "subject": {
                        "type": "string",
                        "description": "Brief subject line for the ticket"
                    },
                    "description": {
                        "type": "string",
                        "description": "Detailed description of the issue"
                    },
                    "priority": {
                        "type": "string",
                        "enum": ["low", "medium", "high", "urgent"],
                        "description": "Ticket priority level"
                    }
                },
                "required": ["subject", "description"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "check_account_info",
            "description": "Look up customer account information",
            "parameters": {
                "type": "object",
                "properties": {
                    "customer_email": {
                        "type": "string",
                        "description": "Customer's email address"
                    }
                },
                "required": ["customer_email"]
            }
        }
    }
]

# Map function names to actual functions
all_functions = {
    "get_order_status": get_order_status,
    "create_support_ticket": create_support_ticket,
    "check_account_info": check_account_info
}

print(f"✅ Defined {len(multi_tools)} tools")

In [ ]:
# Test multi-tool system

test_queries = [
    "What's the status of order ORD-12345?",
    "I need to report a broken laptop. Please create a ticket.",
    "Can you check my account? My email is john@email.com"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"User: {query}")
    print(f"{'='*60}")
    
    messages = [{"role": "user", "content": query}]
    answer = execute_function_call(messages, multi_tools, all_functions)
    
    print(f"🤖 Assistant: {answer}")

## Part 5: Input Validation & Security

Always validate function inputs for security!

In [ ]:
# Validation functions

def validate_order_id(order_id: str) -> bool:
    """Validate order ID format"""
    pattern = r'^ORD-\d{5}$'
    return bool(re.match(pattern, order_id))

def sanitize_text_input(text: str, max_length: int = 500) -> str:
    """Sanitize user text input"""
    
    # Remove leading/trailing whitespace
    text = text.strip()
    
    # Limit length
    if len(text) > max_length:
        text = text[:max_length]
    
    # Remove dangerous characters/patterns
    dangerous_patterns = ['<script>', 'javascript:', 'onerror=', '<?php']
    for pattern in dangerous_patterns:
        if pattern.lower() in text.lower():
            raise ValueError(f"Input contains prohibited pattern: {pattern}")
    
    return text

# Safe versions of functions

def get_order_status_safe(order_id: str) -> dict:
    """Safely get order status with validation"""
    
    # Validate format
    if not validate_order_id(order_id):
        return {
            "error": "Invalid order ID format. Expected: ORD-XXXXX"
        }
    
    # Sanitize input
    order_id = order_id.strip().upper()
    
    # Call original function
    return get_order_status(order_id)

def create_support_ticket_safe(subject: str, description: str, priority: str = "medium") -> dict:
    """Safely create support ticket with validation"""
    
    try:
        # Validate and sanitize inputs
        subject = sanitize_text_input(subject, max_length=100)
        description = sanitize_text_input(description, max_length=1000)
        
        # Validate priority
        valid_priorities = ["low", "medium", "high", "urgent"]
        if priority not in valid_priorities:
            priority = "medium"  # Default fallback
        
        # Call original function
        return create_support_ticket(subject, description, priority)
    
    except ValueError as e:
        return {
            "success": False,
            "error": str(e)
        }

# Test validation
print("Testing validation:")
print("\n1. Valid order ID:")
print(get_order_status_safe("ORD-12345"))

print("\n2. Invalid order ID:")
print(get_order_status_safe("INVALID123"))

print("\n3. Valid ticket:")
print(create_support_ticket_safe("Help needed", "I need assistance", "high"))

## Part 6: SupportGenie v0.4 - Complete Implementation

Putting it all together into a class-based chatbot with function calling.

In [ ]:
class SupportGenieV4:
    """
    SupportGenie Version 0.4
    Now with function calling capabilities!
    """
    
    SYSTEM_MESSAGE = """You are SupportGenie, an AI customer support assistant for TechStore.
    
    You can help customers by:
    - Looking up order status and tracking
    - Creating support tickets for issues
    - Checking account information
    
    Guidelines:
    - Be professional, empathetic, and helpful
    - Use tools when needed to take actions
    - Always confirm after taking an action
    - Ask for clarification if information is missing
    """
    
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)
        self.conversation_history = []
        self.setup_tools()
    
    def setup_tools(self):
        """Define available tools"""
        
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "get_order_status",
                    "description": "Get status and tracking for an order",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "order_id": {
                                "type": "string",
                                "description": "Order ID (ORD-XXXXX)"
                            }
                        },
                        "required": ["order_id"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "create_support_ticket",
                    "description": "Create a support ticket for customer issues",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "subject": {"type": "string"},
                            "description": {"type": "string"},
                            "priority": {
                                "type": "string",
                                "enum": ["low", "medium", "high", "urgent"]
                            }
                        },
                        "required": ["subject", "description"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "check_account_info",
                    "description": "Look up customer account information",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "customer_email": {"type": "string"}
                        },
                        "required": ["customer_email"]
                    }
                }
            }
        ]
        
        self.available_functions = {
            "get_order_status": self.get_order_status,
            "create_support_ticket": self.create_support_ticket,
            "check_account_info": self.check_account_info
        }
    
    def get_order_status(self, order_id: str) -> dict:
        """Mock order lookup"""
        return get_order_status_safe(order_id)
    
    def create_support_ticket(self, subject: str, description: str, priority: str = "medium") -> dict:
        """Mock ticket creation"""
        return create_support_ticket_safe(subject, description, priority)
    
    def check_account_info(self, customer_email: str) -> dict:
        """Mock account lookup"""
        return check_account_info(customer_email)
    
    def chat(self, user_message: str) -> str:
        """Chat with function calling support"""
        
        # Add user message
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        # Build messages with system message
        messages = [
            {"role": "system", "content": self.SYSTEM_MESSAGE}
        ] + self.conversation_history
        
        # Call API with tools
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            tools=self.tools,
            tool_choice="auto"
        )
        
        response_message = response.choices[0].message
        
        # Check for function calls
        if response_message.tool_calls:
            # Add assistant's response to history
            self.conversation_history.append(response_message)
            
            # Execute each function call
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"  📞 Calling {function_name}...")
                
                # Execute function
                function_result = self.available_functions[function_name](**function_args)
                
                # Add result to conversation
                self.conversation_history.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(function_result)
                })
            
            # Get final response
            messages = [
                {"role": "system", "content": self.SYSTEM_MESSAGE}
            ] + self.conversation_history
            
            final_response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages
            )
            
            assistant_message = final_response.choices[0].message.content
        else:
            assistant_message = response_message.content
        
        # Add to history
        self.conversation_history.append({
            "role": "assistant",
            "content": assistant_message
        })
        
        return assistant_message

print("✅ SupportGenie v0.4 class defined")

In [ ]:
# Test SupportGenie v0.4

bot = SupportGenieV4(api_key=os.environ.get('OPENAI_API_KEY'))

print("🤖 SupportGenie v0.4 - Now with Actions!\n")
print("="*60)

# Test 1: Order status
print("\n👤 User: What's the status of order ORD-12345?")
response = bot.chat("What's the status of order ORD-12345?")
print(f"🤖 SupportGenie: {response}")

print("\n" + "="*60)

# Test 2: Create ticket
print("\n👤 User: My laptop screen is cracked. Can you help?")
response = bot.chat("My laptop screen is cracked. Can you help?")
print(f"🤖 SupportGenie: {response}")

print("\n" + "="*60)

# Test 3: Account info
print("\n👤 User: Check my account - jane@email.com")
response = bot.chat("Check my account - jane@email.com")
print(f"🤖 SupportGenie: {response}")

## Exercises

### Exercise 1: Add a Calculator Tool

Create a calculator function that can perform basic math operations.

In [ ]:
# Your code here
def calculator(operation: str, num1: float, num2: float) -> dict:
    """
    Perform basic math operations
    
    Args:
        operation: One of 'add', 'subtract', 'multiply', 'divide'
        num1: First number
        num2: Second number
    """
    # TODO: Implement calculator
    pass

# TODO: Create function schema
# TODO: Test with LLM

### Exercise 2: Add Email Validation

Add email validation to the `check_account_info` function.

In [ ]:
# Your code here
def validate_email(email: str) -> bool:
    """Validate email format"""
    # TODO: Implement email validation
    pass

### Exercise 3: Multi-Step Workflow

Create a scenario where the LLM needs to call multiple functions in sequence.

In [ ]:
# Test multi-step workflow
# Example: "Check my account jane@email.com and then look up order ORD-12345"

# Your code here

## 🎉 Session 4 Complete!

### What You Learned:

✅ Function calling enables LLMs to take actions  
✅ Function schemas tell the LLM when and how to use tools  
✅ Always validate and sanitize function inputs  
✅ Handle errors gracefully  
✅ Multiple tools enable complex workflows  
✅ SupportGenie can now take real actions!

### Next Steps:

In **Session 5: AI Agents with Memory**, you'll learn:
- Agent execution loops (ReAct pattern)
- Memory systems for context
- Autonomous multi-step task completion
- Making SupportGenie truly autonomous!

---

**Continue to**: [Session 5: AI Agents →](05_AI_Agents.ipynb)